In [3]:
# AS usual, a bit of setup
# If you need other libraries, you should import the libraries.
%load_ext autoreload
%autoreload 2
import os, sys
import torch
from torch import Tensor, nn, optim
from torch.nn import functional as F

import torchvision
from torchvision import models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset

import matplotlib.pyplot as plt
import numpy as np
import cv2
import csv

from torch import nn
from torch import optim
from torchvision.transforms.functional import to_pil_image
from torchvision.transforms.functional import pil_to_tensor
import PIL
from matplotlib import cm
from PIL import Image

import glob
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#######################################################
#               Define Transforms
#######################################################

#Array for Augmentations,... (X5 Times of Train Dataset) 

#To Tensor and Normalize as of grayscale data. 
transform_train = []
for num in range(5):
    transform_train.append(transforms.Compose([
        #transforms.RandomCrop(32, padding=4),#[1]
        transforms.RandomRotation(degrees = num * 3 + 3 ),
        transforms.ToTensor(),
        #Normalize 데이터는 recipe를 참고하였음. https://github.com/Armour/pytorch-nn-practice/blob/master/utils/meanstd.py 
        transforms.Normalize((0.5),(0.5)) 
    ]))

# Normalize test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    #Normalize 데이터는 recipe를 참고하였음. https://github.com/Armour/pytorch-nn-practice/blob/master/utils/meanstd.py 
    transforms.Normalize((0.5),(0.5)) 
])

In [ ]:
####################################################
#       Create Train, Valid and Test sets
####################################################

raw_images = [] #to store image in list
raw_answer = [] # to store class values
data = []
path = "/Users/seungjoobaek/Downloads/small_train_data_set"
img_size = 128

with open('/Users/seungjoobaek/Downloads/small_train_data_set/train_data_1.csv') as list:
    reader_obj = csv.reader(list)
    for row in reader_obj:
        try :
            with open(os.path.join(path, row[2]), 'r') as f :
                img_arr = cv2.imread(os.path.join(path, row[2]), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size,img_size))
                raw_images.append(resized_arr)
                raw_answer.append(int(row[3]))
                data.append([resized_arr, row[3]])
                #data.append([torch.Tensor(resized_arr), torch.Tensor(int(row[3]))])
                # 0 => 있다. 
                # 1  > 없다. 
                #data.append(transforms.ToTensor()(resized_arr), row[3]))
        except IOError as e:
            print(e)

In [ ]:
####################################################
#       Add Normal and 폐렴 Data for 1:1:1 ratio
####################################################
normal_path = "/Users/seungjoobaek/Downloads/chest_xray/chest_xray/train/NORMAL"
normalcnt = 0 
for img in os.listdir(normal_path):
        # 이미지 전처리 과정이기에 다양한 문제가 발생할 수 있습니다.
        # try 문을 활용하여 발생하는 문제를 예외처리 합니다.
        try:
            # 이미지를 흑백으로 처리하기 위한 과정 입니다.
            # cv2.imread() : 이미지 파일을 불러옵니다.
            # cv2.IMREAD_GRAYSCALE : 이미지를 흑백으로 처리합니다.
            img_arr = cv2.imread(os.path.join(normal_path, img), cv2.IMREAD_GRAYSCALE)
            resized_arr = cv2.resize(img_arr, (img_size, img_size))

            # 이미지와 이미지의 라벨 정보를 하나로 묶어 data 변수에 할당합니다.
            data.append([resized_arr, "0"])
            normalcnt += 1
            if normalcnt == 1137:
                break
        # 만약 에러가 생기면 어떠한 문제인지를 print() 를 통해 출력합니다.
        except Exception as e:
            print(e)
pneu_path = "/Users/seungjoobaek/Downloads/chest_xray/chest_xray/train/PNEUMONIA"
pneucnt = 0 
for img in os.listdir(pneu_path):
        # 이미지 전처리 과정이기에 다양한 문제가 발생할 수 있습니다.
        # try 문을 활용하여 발생하는 문제를 예외처리 합니다.
        try:
            # 이미지를 흑백으로 처리하기 위한 과정 입니다.
            # cv2.imread() : 이미지 파일을 불러옵니다.
            # cv2.IMREAD_GRAYSCALE : 이미지를 흑백으로 처리합니다.
            img_arr = cv2.imread(os.path.join(pneu_path, img), cv2.IMREAD_GRAYSCALE)
            resized_arr = cv2.resize(img_arr, (img_size, img_size))

            # 이미지와 이미지의 라벨 정보를 하나로 묶어 data 변수에 할당합니다.
            data.append([resized_arr, "2"])
            pneucnt += 1
            if pneucnt == 1537:
                break
        # 만약 에러가 생기면 어떠한 문제인지를 print() 를 통해 출력합니다.
        except Exception as e:
            print(e)
print("updated data size = ", len(data))

In [5]:
####################################################
#       Create Train, Valid and Test sets
####################################################

print('train_image example: ', raw_images[0])
print('class example: ', raw_answer[0])
size = len(data)
print('\nData Size', size, '개')
print('================================')

#2.
# split train valid from train paths (80,10,10)

train_size = int(size * 0.8)
validation_size = int(size * 0.1)
test_size = size - train_size - validation_size

train_dataset, validation_dataset, test_dataset =  random_split(data, [train_size, validation_size, test_size])

print('\n\n Data Example : \n', train_dataset[0])
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_image example:  [[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [ 65  65  69 ... 177 177 173]
 [ 69  69  64 ... 179 177 171]
 [ 69  69  66 ... 182 179 177]]
class example:  1

Data Size 4611 개


 Data Example : 
 [array([[22, 26, 30, ..., 74,  2,  0],
       [37, 44, 72, ..., 79,  1,  0],
       [49, 88, 91, ..., 76,  0,  0],
       ...,
       [ 2,  2, 25, ...,  0,  0,  0],
       [ 2,  5, 35, ...,  0,  0,  0],
       [ 2,  9, 40, ...,  0,  0,  0]], dtype=uint8), '0']
Training Data Size : 3688
Validation Data Size : 461
Testing Data Size : 462


In [6]:
#######################################################
#               Define Dataset Class
#######################################################

class CustomImageDataset(Dataset):
    def __init__(self, data, transform=False):
        self.datas = data
        self.transform = transform
        
    def __len__(self):
        return len(self.datas)

    def __getitem__(self, idx):
        row = self.datas[idx]
        image = row[0]
        label = row[1]
        #image = self.images[idx]

        #label = class_to_idx[label]
        if self.transform is not None:
            image = self.transform(to_pil_image(image))
        
        return image, label
    
#######################################################
#                  Create Dataset
#######################################################

train_datasets = CustomImageDataset(train_dataset,transform_test)
#Augmented Data are added
for transform in transform_train:
    train_datasets += CustomImageDataset(train_dataset,transform)
#others
valid_datasets = CustomImageDataset(validation_dataset,transform_test) #test transforms are applied
test_datasets = CustomImageDataset(test_dataset,transform_test)

In [7]:
print('The shape of tensor for 50th image in train dataset: ',train_datasets[49][0].shape)
print('The label for 50th image in train dataset: ',train_datasets[49][1])
print(len(train_datasets))
cnt = 0
for d in train_datasets :
    if d[1] == "0" or d[1] == 0: #정상 개수
        cnt += 1
print(cnt, cnt/len(train_datasets), len(train_datasets))
cnt = 0
for d in data:
    if d[1] == "0" or d[1] == 0: #정상 개수
        cnt += 1
print(cnt, cnt/len(data), len(data)) #동일 비율 20% 확인. 

The shape of tensor for 50th image in train dataset:  torch.Size([1, 128, 128])
The label for 50th image in train dataset:  0
22128
7410 0.3348698481561822 22128
1536 0.33311646063760575 4611


In [14]:
#######################################################
#                  Define Dataloaders
#######################################################
train_dataloader = DataLoader(train_datasets, batch_size=128, shuffle=True)#, drop_last=True)
validation_dataloader = DataLoader(valid_datasets, batch_size=128, shuffle=True)#, drop_last=True)
test_dataloader = DataLoader(test_datasets, batch_size=128, shuffle=False)#, drop_last=True)

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [16]:
print(len(train_dataloader))

173


In [17]:
from torchvision import models
import torch
# load resnet18 with the pre-trained weights
resnet18_pretrained = models.resnet18(pretrained=True)

def _get_first_layer(m):
    "Access first layer of a model"
    c,p,n = m,None,None  # child, parent, name
    for n in next(m.named_parameters())[0].split('.')[:-1]:
        p,c=c,getattr(c,n)
    return c,p,n

def _load_pretrained_weights(new_layer, previous_layer):
    "Load pretrained weights based on number of input channels"
    n_in = getattr(new_layer, 'in_channels')
    if n_in==1:
        # we take the sum
        new_layer.weight.data = previous_layer.weight.data.sum(dim=1, keepdim=True)
    elif n_in==2:
        # we take first 2 channels + 50%
        new_layer.weight.data = previous_layer.weight.data[:,:2] * 1.5
    else:
        # keep 3 channels weights and set others to null
        new_layer.weight.data[:,:3] = previous_layer.weight.data
        new_layer.weight.data[:,3:].zero_()

def _update_first_layer(model, n_in, pretrained):
    "Change first layer based on number of input channels"
    if n_in == 3: return
    first_layer, parent, name = _get_first_layer(model)
    assert isinstance(first_layer, nn.Conv2d), f'Change of input channels only supported with Conv2d, found {first_layer.__class__.__name__}'
    assert getattr(first_layer, 'in_channels') == 3, f'Unexpected number of input channels, found {getattr(first_layer, "in_channels")} while expecting 3'
    params = {attr:getattr(first_layer, attr) for attr in 'out_channels kernel_size stride padding dilation groups padding_mode'.split()}
    params['bias'] = getattr(first_layer, 'bias') is not None
    params['in_channels'] = n_in
    new_layer = nn.Conv2d(**params)
    if pretrained:
        _load_pretrained_weights(new_layer, first_layer)
    setattr(parent, name, new_layer)

renet18_pretrained = _update_first_layer(resnet18_pretrained, 1,True)
print(resnet18_pretrained)

# change the output layer to 2 classes
num_classes = 3 # 0 - 정상 , 1 - 기흉 ,  2 - 폐렴
num_ftrs = resnet18_pretrained.fc.in_features
resnet18_pretrained.fc = nn.Linear(num_ftrs, num_classes)

net = resnet18_pretrained.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
loss_fun = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [22]:
# Train the model
epochs = 12
  # number of epochs

history = []
for epoch in range(epochs):

    loss_tmp = 0.0
    epoch_loss = 0.0 
    
    for i, datas in enumerate(train_dataloader, start=0):
        # Load the data
        inputs = datas[0]
        labels = [int(a) for a in datas[1]] #넣어진 데이터가 str 이어서 int로 바꿔봄. 
        labels = torch.tensor(labels) 

        inputs = inputs.to(device)
        labels = labels.to(device)

        # Estimate the output using the network
        outputs = net(inputs)

        # Calculate the loss between the output of the network and label
        loss = loss_fun(outputs, labels)

        # Optimize the network 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_tmp += loss.data
        epoch_loss += loss.data
        if i % 30 == 29:    # Print loss every 5000 mini-batches
            print('[Epoch - %d, Iteration - %5d] Loss: %.3f' %(epoch + 1, i + 1, loss_tmp / (i+1)))
            loss_tmp = 0.0
    
    # Update the learning rate according to the learnig rate scheduler
    scheduler.step()
    
    net.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for item in validation_dataloader:
            inputs = item[0]
            labels = [int(a) for a in item[1]]
            labels = torch.tensor(labels)
    
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += len(labels)
            correct += (predicted == labels).sum().item()
        history.append(100 * correct / total)
        print('Check Accuracy of the model on the {} valid images: {} % at epoch : {}'.format(total, 100 * correct / total, epoch+1))
        


    # Print the epoch loss
    print('[Epoch - %d] Loss: %.3f' %(epoch + 1, epoch_loss / (i+1)))

print('Finished Training')

[Epoch - 1, Iteration -    30] Loss: 0.895
[Epoch - 1, Iteration -    60] Loss: 0.363
[Epoch - 1, Iteration -    90] Loss: 0.193
[Epoch - 1, Iteration -   120] Loss: 0.107
[Epoch - 1, Iteration -   150] Loss: 0.084
Check Accuracy of the model on the 461 valid images: 86.11713665943601 % at epoch : 1
[Epoch - 1] Loss: 0.581
[Epoch - 2, Iteration -    30] Loss: 0.348
[Epoch - 2, Iteration -    60] Loss: 0.164
[Epoch - 2, Iteration -    90] Loss: 0.125
[Epoch - 2, Iteration -   120] Loss: 0.084
[Epoch - 2, Iteration -   150] Loss: 0.067
Check Accuracy of the model on the 461 valid images: 89.587852494577 % at epoch : 2
[Epoch - 2] Loss: 0.339
[Epoch - 3, Iteration -    30] Loss: 0.302
[Epoch - 3, Iteration -    60] Loss: 0.137
[Epoch - 3, Iteration -    90] Loss: 0.091
[Epoch - 3, Iteration -   120] Loss: 0.067
[Epoch - 3, Iteration -   150] Loss: 0.054
Check Accuracy of the model on the 461 valid images: 88.93709327548807 % at epoch : 3
[Epoch - 3] Loss: 0.280
[Epoch - 4, Iteration -    

In [ ]:
net.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
correct = {0 : 0, 1 : 0, 2 : 0}
with torch.no_grad():
    total_correct = 0
    total = 0
    for item in test_dataloader:
        inputs = item[0]
        labels = [int(a) for a in item[1]]
        labels = torch.tensor(labels)

        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        
        labels = [int(a) for a in labels]
        predicted = [int(a) for a in predicted]
        for answers in range(len(labels)):
            if predicted[answers] == labels[answers] :
                correct[labels[answers]] += 1
                total_correct += 1
        #correct[int(labels)] += (predicted == labels).sum().item()
        #correct += (predicted == labels).sum().item()
    print('Test Accuracy for Each Class : ')
    clslabel = {0 : "정상", 1:"기흉", 2:"폐렴"}
    
    for cls, cnt in correct.items() : 
        print('{} 을 맞춘 정확도 : {} %'.format(clslabel[cls], 100 * cnt * 3 / total))
    print(' 종합 정확도 {}%'.format(total_correct / total))
    #print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / (total/3)))
print(correct)

In [29]:
print(history)

[86.11713665943601, 89.587852494577, 88.93709327548807, 90.02169197396964, 89.3709327548807, 89.587852494577, 86.98481561822126, 89.587852494577, 89.15401301518438, 89.3709327548807, 89.80477223427332, 89.3709327548807]
